Note: I left the code to generate the htfs for each test system in the notebook, but if you don't want to re-generate, you can use the pickled htf that I saved (I left the full path in for you). I generated these htfs based after you added support for `lambda_{i}_bonds_old/new` and `scale_lambda_0_bonds`

In [1]:
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from simtk import unit, openmm
from perses.tests.utils import compute_potential_components
from openmmtools.constants import kB
from perses.dispersed.utils import configure_platform
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
import numpy as np
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.testsystems import AlanineDipeptideVacuum, AlanineDipeptideExplicit
import itertools
from perses.tests.test_topology_proposal import generate_dipeptide_top_pos_sys, generate_atp
import pickle

#############################################
# CONSTANTS
#############################################
temperature = 298.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
REFERENCE_PLATFORM = openmm.Platform.getPlatformByName("CUDA")


conducting subsequent work with the following platform: CPU


INFO:rdkit:Enabling RDKit 2021.03.1 jupyter extensions


In [2]:
def test_bond_energies(htf, is_old=True, is_solvated=False):
    # Get harmonic bond force and old/new positions
    system = htf._topology_proposal.old_system if is_old else htf._topology_proposal.new_system
    harmonic_bond_force = system.getForce(0) 
    positions = htf.old_positions(htf.hybrid_positions) if is_old else htf.new_positions(htf.hybrid_positions)
    
    # Get custom bond force and hybrid positions
    bond_force_index = 1 if is_solvated else 0
    hybrid_system = htf.hybrid_system
    custom_bond_force = hybrid_system.getForce(bond_force_index)
    hybrid_positions = htf.hybrid_positions
    
    # Remove nonbonded/exception forces
    for i in range(hybrid_system.getNumForces(), hybrid_system.getNumForces() - 4, -1):
        hybrid_system.removeForce(i - 1)
    
    # Set global parameters
    lambda_old = 1 if is_old else 0
    lambda_new = 0 if is_old else 1
    for i in range(custom_bond_force.getNumGlobalParameters()):
        if custom_bond_force.getGlobalParameterName(i) == 'lambda_0_bonds_old':
            custom_bond_force.setGlobalParameterDefaultValue(i, lambda_old)
        if custom_bond_force.getGlobalParameterName(i) == 'lambda_0_bonds_new':
            custom_bond_force.setGlobalParameterDefaultValue(i, lambda_new)
            
#     for hybrid_idx, idx in htf._hybrid_to_environment_bond_indices.items():
#         p1, p2, length, k = harmonic_bond_force.getBondParameters(idx)
#         harmonic_bond_force.setBondParameters(hybrid_idx, p1, p2, length, k*0)
        
    # Zero the unique old/new bonds in the custom bond force
    hybrid_to_bond_indices = htf._hybrid_to_new_bond_indices if is_old else htf._hybrid_to_old_bond_indices
    for hybrid_idx, idx in hybrid_to_bond_indices.items():
        p1, p2, hybrid_params = custom_bond_force.getBondParameters(hybrid_idx)
        hybrid_params = list(hybrid_params)
        index_to_zero = -3 if is_old else -1
        hybrid_params[index_to_zero] *= 0
        custom_bond_force.setBondParameters(hybrid_idx, p1, p2, hybrid_params)

#     # Zero the unique old/new bonds in the custom bond force
#     hybrid_to_bond_indices = htf._hybrid_to_environment_bond_indices
#     for hybrid_idx, idx in hybrid_to_bond_indices.items():
#         p1, p2, hybrid_params = custom_bond_force.getBondParameters(hybrid_idx)
#         hybrid_params = list(hybrid_params)
# #         index_to_zero = -1 if is_old else -3
#         hybrid_params[-1] *= 0
#         hybrid_params[-2] *= 0
    
            
#         custom_bond_force.setBondParameters(hybrid_idx, p1, p2, hybrid_params)

    ## Get energy components of standard bond force
    platform = configure_platform(REFERENCE_PLATFORM)
    thermostate_other = ThermodynamicState(system=system, temperature=temperature)
    integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
    context_other = thermostate_other.create_context(integrator_other)
    context_other.setPositions(positions)
    components_other = compute_potential_components(context_other, beta=beta)
    print(components_other)
    
    ## Get energy components of custom bond force
    if is_solvated:
        custom_bond_force.setUsesPeriodicBoundaryConditions(True)
    thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
    integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
    context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
    context_hybrid.setPositions(hybrid_positions)
    components_hybrid = compute_potential_components(context_hybrid, beta=beta)
    print(components_hybrid)
    
    assert np.isclose([components_other[0][1]], [components_hybrid[0][1]])
    
    print("Success! Custom bond force and standard bond force energies are equal!")

# Alanine dipeptide in vacuum

In [3]:
# Create a htf
atp, system_generator = generate_atp(phase = 'vacuum')
htf = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                                rxn_field=True,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False,
                                conduct_htf_prop=True)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}
INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
IN

making topology proposal
generating geometry engine
making geometry proposal from ALA to THR
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 8 new atoms
INFO:geometry:	reduced angle potential = 0.0017554373207463183.
INFO:geometry:	reduced angle potential = 9.309346877780187e-05.
INFO:geometry:	reduced angle potential = 0.41923726982898996.
INFO:geometry:	reduced angle potential = 0.0015268165627356017.
INFO:geometry:	reduced angle potential = 0.270688709647723.
INFO:geometry:	reduced angle potential = 0.005809599699452005.
INFO:geometry:	reduced angle potential = 0.829385223112024.
INFO:geometry:	reduced angle potential = 0.6403645744580588.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 11 bond forces in the no-nonbonded final system
INFO:geometry:	there are 43 an

...

INFO:geometry:total reduced potential before atom placement: 9.11648114171204


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 20.672986672125617
INFO:geometry:final reduced energy 29.78946596595154
INFO:geometry:sum of energies: 29.789467813837657
INFO:geometry:magnitude of difference in the energies: 1.8478861179005435e-06
INFO:geometry:Final logp_proposal: -27149.810306568783


added energy components: [('CustomBondForce', 0.000520203927326505), ('CustomAngleForce', 0.4511197756248138), ('CustomTorsionForce', 7.250463328533253), ('CustomBondForce', 12.970883364040223)]


INFO:relative:*** Generating RxnHybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:scale_templates: [['nonscale_lambda'], ['nonscale_region']]
INFO:relative:particle (nonbonded) scale_templates: [['nonscale_lambda'], ['nonscale_region1'], ['nonscale_region2']]
INFO:relative:No MonteCarloBarostat added.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quan

In [4]:
with open("atp_vacuum.pickle", "wb") as f:
    pickle.dump(htf, f)

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [2]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [6]:
test_bond_energies(htf)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478365356521381), ('HarmonicAngleForce', 0.6112775462128995), ('PeriodicTorsionForce', 16.285398289983064), ('NonbondedForce', -39.4499277456003), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.03478365356521381), ('CustomAngleForce', 3.8580778529179494), ('CustomTorsionForce', 16.285403285921724), ('CustomNonbondedForce', -135.30622366805557), ('CustomNonbondedForce', 4.748478747607475), ('CustomBondForce', 82.63803400070168), ('CustomBondForce', 8.469780004620791), ('AndersenThermostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [7]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [8]:
test_bond_energies(htf, is_old=False)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 1.2957330745733426), ('HarmonicAngleForce', 3.403930427792299), ('PeriodicTorsionForce', 30.268405909483896), ('NonbondedForce', 388.51299936325097), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 1.2957330745733426), ('CustomAngleForce', 3.8580778529179494), ('CustomTorsionForce', 16.285403285921724), ('CustomNonbondedForce', -135.30622366805557), ('CustomNonbondedForce', 4.748478747607475), ('CustomBondForce', 82.63803400070168), ('CustomBondForce', 8.469780004620791), ('AndersenThermostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


### Test one alchemical region, one scale region

In [9]:
# Create a htf
atp, system_generator = generate_atp(phase = 'vacuum')
htf = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                                rxn_field=True,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False,
                                conduct_htf_prop=True, 
                                scale_regions=[[10, 11, 12, 13, 22, 23, 24, 25, 26, 27, 28, 29]])

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}
INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
IN

making topology proposal
generating geometry engine
making geometry proposal from ALA to THR
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 8 new atoms
INFO:geometry:	reduced angle potential = 0.2755483094524574.
INFO:geometry:	reduced angle potential = 0.12663277410293056.
INFO:geometry:	reduced angle potential = 0.6734407664093444.
INFO:geometry:	reduced angle potential = 0.3556657471824796.
INFO:geometry:	reduced angle potential = 1.8210633634401288.
INFO:geometry:	reduced angle potential = 0.10657757997174908.
INFO:geometry:	reduced angle potential = 0.01449024279734101.
INFO:geometry:	reduced angle potential = 0.05148990975102987.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 11 bond forces in the no-nonbonded final system
INFO:geometry:	there are 43 angle fo

...

INFO:geometry:total reduced potential before atom placement: 9.11648114171204


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 20.672986672125617
INFO:geometry:final reduced energy 29.78946596595154
INFO:geometry:sum of energies: 29.789467813837657
INFO:geometry:magnitude of difference in the energies: 1.8478861179005435e-06
INFO:geometry:Final logp_proposal: -26840.97686061595


added energy components: [('CustomBondForce', 0.000520203927326505), ('CustomAngleForce', 0.4511197756248138), ('CustomTorsionForce', 7.250463328533253), ('CustomBondForce', 12.970883364040223)]


INFO:relative:*** Generating RxnHybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:scale_templates: [['nonscale_lambda', 'scale_lambda_0', 'interscale_lambda_0'], ['nonscale_region', 'scale_region_0', 'interscale_region_0']]
INFO:relative:particle (nonbonded) scale_templates: [['nonscale_lambda', 'scale_lambda_0'], ['nonscale_region1', 'scale_region_01'], ['nonscale_region2', 'scale_region_02']]
INFO:relative:No MonteCarloBa

In [10]:
with open("atp_vacuum_scale_region.pickle", "wb") as f:
    pickle.dump(htf, f)

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [3]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [14]:
htf.hybrid_system.getForces()

[<simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2aca5c9e0d50> >,
 <simtk.openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2aca5c9e0300> >,
 <simtk.openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2aca5c9e0270> >,
 <simtk.openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2aca5c9e01b0> >,
 <simtk.openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2aca5c9e0060> >,
 <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2aca5c9e0a80> >,
 <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2aca5c9e0c00> >]

In [17]:
htf.hybrid_system.getForce(2).getEnergyFunction()

'( nonscale_lambda_torsions * nonscale_region+scale_lambda_0_torsions * scale_region_0+interscale_lambda_0_torsions * interscale_region_0 ) * (( 1 * environment_region+lambda_0_torsions_old * alchemical_region_0 ) * U1 + ( 1 * environment_region+lambda_0_torsions_new * alchemical_region_0 ) * U2);U1 = K1*(1+cos(periodicity1*theta-phase1));U2 = K2*(1+cos(periodicity2*theta-phase2));'

In [10]:
for i in range(htf.hybrid_system.getForce(0).getNumGlobalParameters()):
    print(htf.hybrid_system.getForce(0).getGlobalParameterName(i))

lambda_0_bonds_old
lambda_0_bonds_new
nonscale_lambda_bonds
scale_lambda_0_bonds
interscale_lambda_0_bonds


In [11]:
for i in range(htf.hybrid_system.getForce(0).getNumPerBondParameters()):
    print(htf.hybrid_system.getForce(0).getPerBondParameterName(i))

nonscale_region
scale_region_0
interscale_region_0
environment_region
alchemical_region_0
length1
K1
length2
K2


In [15]:
for i in range(htf.hybrid_system.getForce(1).getNumPerAngleParameters()):
    print(htf.hybrid_system.getForce(1).getPerAngleParameterName(i))

nonscale_region
scale_region_0
interscale_region_0
environment_region
alchemical_region_0
theta0_1
K1
theta0_2
K2


In [12]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[10, 11, 12, 13, 22, 23, 24, 25, 26, 27, 28, 29]

In [13]:
test_bond_energies(htf)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478365356521381), ('HarmonicAngleForce', 0.6112775462128995), ('PeriodicTorsionForce', 16.285398289983064), ('NonbondedForce', -39.4499277456003), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.03478365356521381), ('CustomAngleForce', 12.79191873055503), ('CustomTorsionForce', 16.285403285921724), ('CustomNonbondedForce', -135.30622366805557), ('CustomNonbondedForce', 4.748478747607475), ('CustomBondForce', 82.63803400070168), ('CustomBondForce', 8.469780004620791), ('AndersenThermostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [14]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [15]:
test_bond_energies(htf, is_old=False)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 3.0804088330039257), ('HarmonicAngleForce', 12.337771305429378), ('PeriodicTorsionForce', 28.091644511264406), ('NonbondedForce', 56.74765841438245), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 3.0804088330039257), ('CustomAngleForce', 12.79191873055503), ('CustomTorsionForce', 16.285403285921724), ('CustomNonbondedForce', -135.30622366805557), ('CustomNonbondedForce', 4.748478747607475), ('CustomBondForce', 82.63803400070168), ('CustomBondForce', 8.469780004620791), ('AndersenThermostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


# Alanine dipeptide in solvent

In [16]:
# Create a htf
atp, system_generator = generate_atp(phase = 'solvent')
htf = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                                rxn_field=True,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False,
                                conduct_htf_prop=True)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}
INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
IN

making topology proposal


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 8
INFO:geometry:Atom index proposal order is [10, 18, 14, 17, 16, 13, 19, 15]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...


generating geometry engine
making geometry proposal from ALA to THR


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 11 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 43 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 72 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 1654 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (7, 6, 8, 10, [1, Quantity(value=0.99000

...

INFO:geometry:total reduced potential before atom placement: 9.11648114171204


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 20.672986672125617
INFO:geometry:final reduced energy 29.78946596595156
INFO:geometry:sum of energies: 29.789467813837657
INFO:geometry:magnitude of difference in the energies: 1.8478860965842614e-06
INFO:geometry:Final logp_proposal: -26895.200283713013


added energy components: [('CustomBondForce', 0.000520203927326505), ('CustomAngleForce', 0.4511197756248138), ('CustomTorsionForce', 7.250463328533253), ('CustomBondForce', 12.970883364040223)]


INFO:relative:*** Generating RxnHybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:scale_templates: [['nonscale_lambda'], ['nonscale_region']]
INFO:relative:particle (nonbonded) scale_templates: [['nonscale_lambda'], ['nonscale_region1'], ['nonscale_region2']]
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=

In [17]:
with open("atp_solvent.pickle", "wb") as f:
    pickle.dump(htf, f)

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [27]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [28]:
test_bond_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.034783653565214685), ('HarmonicAngleForce', 0.6112775462129015), ('PeriodicTorsionForce', 16.285398289983082), ('NonbondedForce', -1017.9419702202207), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.03478365356521381), ('CustomAngleForce', 9.27903555849094), ('CustomTorsionForce', 16.285403285921724), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [29]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [30]:
test_bond_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 2.108235239917542), ('HarmonicAngleForce', 8.824888133365281), ('PeriodicTorsionForce', 25.495955289982533), ('NonbondedForce', -1054.9212696042337), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 2.108235239917472), ('CustomAngleForce', 9.27903555849094), ('CustomTorsionForce', 16.285403285921724), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


### Test one alchemical region, one scale region

In [31]:
# Create a htf
atp, system_generator = generate_atp(phase = 'solvent')
htf = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                                rxn_field=True,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False,
                                conduct_htf_prop=True, 
                                scale_regions=[[10, 11, 12, 13, 1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564]])

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}
INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
IN

making topology proposal


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 8
INFO:geometry:Atom index proposal order is [10, 18, 14, 16, 13, 17, 19, 15]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...


generating geometry engine
making geometry proposal from ALA to THR


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 11 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 43 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 72 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 1654 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (7, 6, 8, 10, [1, Quantity(value=0.99000

...

INFO:geometry:total reduced potential before atom placement: 9.11648114171204


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 20.672986672125617
INFO:geometry:final reduced energy 29.78946596595156
INFO:geometry:sum of energies: 29.789467813837657
INFO:geometry:magnitude of difference in the energies: 1.8478860965842614e-06
INFO:geometry:Final logp_proposal: -26852.09238988592


added energy components: [('CustomBondForce', 0.000520203927326505), ('CustomAngleForce', 0.4511197756248138), ('CustomTorsionForce', 7.250463328533253), ('CustomBondForce', 12.970883364040223)]


INFO:relative:*** Generating RxnHybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:scale_templates: [['nonscale_lambda', 'scale_lambda_0', 'interscale_lambda_0'], ['nonscale_region', 'scale_region_0', 'interscale_region_0']]
INFO:relative:particle (nonbonded) scale_templates: [['nonscale_lambda', 'scale_lambda_0'], ['nonscale_region1', 'scale_region_01'], ['nonscale_region2', 'scal

In [32]:
with open("atp_solvent_scale_region.pickle", "wb") as f:
    pickle.dump(htf, f)

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [5]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [6]:
htf.hybrid_system.getForce(0).getEnergyFunction()

'( nonscale_lambda_bonds * nonscale_region+scale_lambda_0_bonds * scale_region_0+interscale_lambda_0_bonds * interscale_region_0 ) * (K/2)*(r-length)^2;K = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * K1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * K2;length = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * length1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * length2;'

In [34]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[10, 11, 12, 13, 1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564]

In [35]:
test_bond_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.034783653565214685), ('HarmonicAngleForce', 0.6112775462129015), ('PeriodicTorsionForce', 16.285398289983082), ('NonbondedForce', -995.1493033223794), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.03478365356521381), ('CustomAngleForce', 9.661801162668427), ('CustomTorsionForce', 16.285403285921724), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [50]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [37]:
test_bond_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.9148868518127816), ('HarmonicAngleForce', 9.207653737542962), ('PeriodicTorsionForce', 27.875973709878817), ('NonbondedForce', -940.6669304074791), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.9148868518127886), ('CustomAngleForce', 9.661801162668427), ('CustomTorsionForce', 16.285403285921724), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


# CDK2 transformation in solvent

In [4]:
import os
from pkg_resources import resource_filename
from perses.app import setup_relative_calculation
import pickle

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


In [5]:
# Create a htf
setup_directory = resource_filename("perses", "data/cdk2-example")

# Get options
from perses.app.setup_relative_calculation import getSetupOptions
yaml_filename = os.path.join(setup_directory, "cdk2_setup_repex.yaml")
setup_options = getSetupOptions(yaml_filename)

# Update options
for parameter in ['protein_pdb', 'ligand_file']:
    setup_options[parameter] = os.path.join(setup_directory, setup_options[parameter])
    
setup_options['rxn_field'] = True
setup_options['phases'] = ['solvent']
setup_options['scale_regions'] = None
setup_options['validate_endstate_energies'] = False
    
setup_dict = setup_relative_calculation.run_setup(setup_options, serialize_systems=False, build_samplers=False)

2021-07-14 16:27:57,425:(0.00s):root:	Detecting phases...
2021-07-14 16:27:57,428:(0.00s):root:		phases detected: ['complex', 'solvent', 'vacuum']
2021-07-14 16:27:57,430:(0.00s):root:No constraints will be removed
2021-07-14 16:27:57,433:(0.00s):root:No spectators
2021-07-14 16:27:57,435:(0.00s):root:			run_type is not specified; default to None
2021-07-14 16:27:57,438:(0.00s):root:	Detecting fe_type...
2021-07-14 16:27:57,440:(0.00s):root:		fe_type: repex
2021-07-14 16:27:57,442:(0.00s):root:			offline-freq not specified: default to 10.
2021-07-14 16:27:57,445:(0.00s):root:	'neglect_angles' detected: False.
2021-07-14 16:27:57,447:(0.00s):root:	'softcore_v2' not specified: default to 'False'
2021-07-14 16:27:57,450:(0.00s):root:	Creating 'output'...
2021-07-14 16:27:57,454:(0.00s):root:	detected ligand file: /home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.9.0+78.gee97a4e.dirty-py3.8.egg/perses/data/cdk2-example/CDK2_ligands.sdf
2021-07-14 16:27:57,4

2021-07-14 16:28:03,112:(0.00s):geometry:Conducting forward proposal...
2021-07-14 16:28:03,114:(0.00s):geometry:Computing proposal order with NetworkX...
2021-07-14 16:28:03,132:(0.02s):geometry:number of atoms to be placed: 1
2021-07-14 16:28:03,135:(0.00s):geometry:Atom index proposal order is [2190]
2021-07-14 16:28:03,137:(0.00s):geometry:omitted_bonds: []
2021-07-14 16:28:03,140:(0.00s):geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
2021-07-14 16:28:03,461:(0.32s):geometry:creating growth system...
2021-07-14 16:28:03,541:(0.08s):geometry:	creating bond force...
2021-07-14 16:28:03,544:(0.00s):geometry:	there are 27 bonds in reference force.
2021-07-14 16:28:03,547:(0.00s):geometry:	creating angle force...
2021-07-14 16:28:03,549:(0.00s):geometry:	there are 85 angles in reference force.
2021-07-14 16:28:03,554:(0.00s):geometry:	creating torsion force...
2021-07-14 16:28:03,556:(0.00s):geometry:	creating extra

conducting subsequent work with the following platform: CUDA


2021-07-14 16:28:04,273:(0.59s):geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


...

2021-07-14 16:28:11,181:(1.20s):geometry:total reduced potential before atom placement: 48.41899607369698


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-07-14 16:28:12,578:(1.40s):geometry:total reduced energy added from growth system: -5.056837038117532
2021-07-14 16:28:12,581:(0.00s):geometry:final reduced energy 43.36216011253515
2021-07-14 16:28:12,584:(0.00s):geometry:sum of energies: 43.36215903557945
2021-07-14 16:28:12,587:(0.00s):geometry:magnitude of difference in the energies: 1.0769557015066766e-06
2021-07-14 16:28:12,589:(0.00s):geometry:Final logp_proposal: 11.56594796641107


added energy components: [('CustomBondForce', 0.0879842954280575), ('CustomAngleForce', 0.04605409749261916), ('CustomTorsionForce', 0.005423200139534058), ('CustomBondForce', -5.1962986311777435)]


2021-07-14 16:28:12,773:(0.18s):root:	writing pickle output...
2021-07-14 16:28:12,992:(0.22s):root:	successfully dumped pickle.
2021-07-14 16:28:12,998:(0.01s):root:	setup is complete.  Writing proposals and positions for each phase to top_prop dict...
2021-07-14 16:28:13,001:(0.00s):root:	writing atom_mapping.png
2021-07-14 16:28:13,187:(0.19s):root:	 steps per move application: 50
2021-07-14 16:28:13,191:(0.00s):root:	trajectory prefix: out
2021-07-14 16:28:13,194:(0.00s):root:	atom selection detected: not water
2021-07-14 16:28:13,196:(0.00s):root:	no nonequilibrium detected.
2021-07-14 16:28:13,199:(0.00s):root:	cataloging HybridTopologyFactories...
2021-07-14 16:28:13,201:(0.00s):root:		phase: solvent:
2021-07-14 16:28:13,204:(0.00s):root:		writing HybridTopologyFactory for phase solvent...
2021-07-14 16:28:13,206:(0.00s):relative:*** Generating RxnHybridTopologyFactory ***
2021-07-14 16:28:13,209:(0.00s):relative:Beginning nonbonded method, total particle, barostat, and exceptio

In [6]:
htf = setup_dict['hybrid_topology_factories']['solvent']

In [7]:
with open("cdk2_solvent.pickle", "wb") as f:
    pickle.dump(htf, f)

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [8]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [9]:
htf.hybrid_system.getForces()

[<simtk.openmm.openmm.MonteCarloBarostat; proxy of <Swig Object of type 'OpenMM::MonteCarloBarostat *' at 0x2ae7391d6d50> >,
 <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2ae7391d6e10> >,
 <simtk.openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2ae7391d6e70> >,
 <simtk.openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2ae7391d6f30> >,
 <simtk.openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2ae7391d6fc0> >,
 <simtk.openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2ae731198090> >,
 <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2ae731198120> >,
 <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2ae7311981b0> >]

In [10]:
test_bond_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.608082977954756), ('HarmonicAngleForce', 29.199523609209102), ('PeriodicTorsionForce', 15.076747992617829), ('NonbondedForce', 3350.162471334583), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 31.450793105256693), ('CustomTorsionForce', 15.076753795301011), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [11]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [12]:
test_bond_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.519508183899664), ('HarmonicAngleForce', 31.40442991986144), ('PeriodicTorsionForce', 15.078153431480851), ('NonbondedForce', 3363.615071727043), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.519508183899664), ('CustomAngleForce', 31.450793105256693), ('CustomTorsionForce', 15.076753795301011), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


### Test one alchemical region, one scale region

In [14]:
import os
from pkg_resources import resource_filename
from perses.app import setup_relative_calculation
import pickle

In [15]:
# Create a htf
setup_directory = resource_filename("perses", "data/cdk2-example")

# Get options
from perses.app.setup_relative_calculation import getSetupOptions
yaml_filename = os.path.join(setup_directory, "cdk2_setup_repex.yaml")
setup_options = getSetupOptions(yaml_filename)

# Update options
for parameter in ['protein_pdb', 'ligand_file']:
    setup_options[parameter] = os.path.join(setup_directory, setup_options[parameter])
    
setup_options['rxn_field'] = True
setup_options['phases'] = ['solvent']
setup_options['scale_regions'] = [[44, 2197]]
setup_options['validate_endstate_energies'] = False
    
setup_dict = setup_relative_calculation.run_setup(setup_options, serialize_systems=False, build_samplers=False)

2021-07-14 16:28:32,850:(18.12s):root:	Detecting phases...
2021-07-14 16:28:32,854:(0.00s):root:		phases detected: ['complex', 'solvent', 'vacuum']
2021-07-14 16:28:32,857:(0.00s):root:No constraints will be removed
2021-07-14 16:28:32,859:(0.00s):root:No spectators
2021-07-14 16:28:32,862:(0.00s):root:			run_type is not specified; default to None
2021-07-14 16:28:32,864:(0.00s):root:	Detecting fe_type...
2021-07-14 16:28:32,867:(0.00s):root:		fe_type: repex
2021-07-14 16:28:32,869:(0.00s):root:			offline-freq not specified: default to 10.
2021-07-14 16:28:32,872:(0.00s):root:	'neglect_angles' detected: False.
2021-07-14 16:28:32,874:(0.00s):root:	'softcore_v2' not specified: default to 'False'
2021-07-14 16:28:32,876:(0.00s):root:	Creating 'output'...
2021-07-14 16:28:32,885:(0.01s):root:	detected ligand file: /home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.9.0+78.gee97a4e.dirty-py3.8.egg/perses/data/cdk2-example/CDK2_ligands.sdf
2021-07-14 16:28:32,

2021-07-14 16:28:38,851:(0.00s):geometry:Conducting forward proposal...
2021-07-14 16:28:38,854:(0.00s):geometry:Computing proposal order with NetworkX...
2021-07-14 16:28:38,872:(0.02s):geometry:number of atoms to be placed: 1
2021-07-14 16:28:38,876:(0.00s):geometry:Atom index proposal order is [2190]
2021-07-14 16:28:38,878:(0.00s):geometry:omitted_bonds: []
2021-07-14 16:28:38,881:(0.00s):geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
2021-07-14 16:28:39,416:(0.53s):geometry:creating growth system...
2021-07-14 16:28:39,498:(0.08s):geometry:	creating bond force...
2021-07-14 16:28:39,502:(0.00s):geometry:	there are 27 bonds in reference force.
2021-07-14 16:28:39,505:(0.00s):geometry:	creating angle force...
2021-07-14 16:28:39,507:(0.00s):geometry:	there are 85 angles in reference force.
2021-07-14 16:28:39,511:(0.00s):geometry:	creating torsion force...
2021-07-14 16:28:39,514:(0.00s):geometry:	creating extra

conducting subsequent work with the following platform: CUDA


2021-07-14 16:28:40,291:(0.65s):geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


...

2021-07-14 16:28:47,139:(1.21s):geometry:total reduced potential before atom placement: 48.41899607369698


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-07-14 16:28:48,540:(1.40s):geometry:total reduced energy added from growth system: -5.056837038117532
2021-07-14 16:28:48,543:(0.00s):geometry:final reduced energy 43.36216011253515
2021-07-14 16:28:48,546:(0.00s):geometry:sum of energies: 43.36215903557945
2021-07-14 16:28:48,548:(0.00s):geometry:magnitude of difference in the energies: 1.0769557015066766e-06
2021-07-14 16:28:48,550:(0.00s):geometry:Final logp_proposal: 11.821616720480304


added energy components: [('CustomBondForce', 0.0879842954280575), ('CustomAngleForce', 0.04605409749261916), ('CustomTorsionForce', 0.005423200139534058), ('CustomBondForce', -5.1962986311777435)]


2021-07-14 16:28:48,726:(0.18s):root:	writing pickle output...
2021-07-14 16:28:48,942:(0.22s):root:	successfully dumped pickle.
2021-07-14 16:28:48,947:(0.01s):root:	setup is complete.  Writing proposals and positions for each phase to top_prop dict...
2021-07-14 16:28:48,950:(0.00s):root:	writing atom_mapping.png
2021-07-14 16:28:49,031:(0.08s):root:	 steps per move application: 50
2021-07-14 16:28:49,034:(0.00s):root:	trajectory prefix: out
2021-07-14 16:28:49,037:(0.00s):root:	atom selection detected: not water
2021-07-14 16:28:49,039:(0.00s):root:	no nonequilibrium detected.
2021-07-14 16:28:49,041:(0.00s):root:	cataloging HybridTopologyFactories...
2021-07-14 16:28:49,043:(0.00s):root:		phase: solvent:
2021-07-14 16:28:49,046:(0.00s):root:		writing HybridTopologyFactory for phase solvent...
2021-07-14 16:28:49,048:(0.00s):relative:*** Generating RxnHybridTopologyFactory ***
2021-07-14 16:28:49,050:(0.00s):relative:Beginning nonbonded method, total particle, barostat, and exceptio

In [16]:
htf = setup_dict['hybrid_topology_factories']['solvent']

In [17]:
with open("cdk2_solvent_scale_region.pickle", "wb") as f:
    pickle.dump(htf, f)

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [18]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [19]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[44, 2197]

In [20]:
test_bond_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.608082977954756), ('HarmonicAngleForce', 29.199523609209102), ('PeriodicTorsionForce', 15.076747992617829), ('NonbondedForce', 1789.8818680576826), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 30.17311292378729), ('CustomTorsionForce', 15.076753795301011), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [21]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [22]:
test_bond_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.519508183899664), ('HarmonicAngleForce', 30.12674973839204), ('PeriodicTorsionForce', 15.078888310009113), ('NonbondedForce', 1805.061274837831), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.519508183899664), ('CustomAngleForce', 30.17311292378729), ('CustomTorsionForce', 15.076753795301011), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


# 8mer in solvent

In [23]:
import os
from pkg_resources import resource_filename
from perses.app import setup_relative_calculation
import pickle
from simtk import openmm, unit
from simtk.openmm import app
from openmmforcefields.generators import SystemGenerator
import numpy as np

In [24]:
# Test 8-mer peptide in solvent
peptide_filename = resource_filename('perses', 'data/8mer-example/4zuh_peptide_capped.pdb')
pdb = app.PDBFile(peptide_filename)
forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
# barostat = None
modeller = app.Modeller(pdb.topology, pdb.positions)
 
system_generator = SystemGenerator(forcefields=forcefield_files,
                                   barostat=barostat,
                                   forcefield_kwargs={'removeCMMotion': False,
                                                      'ewaldErrorTolerance': 0.00025,
                                                      'constraints': app.HBonds,
                                                      'hydrogenMass': 4 * unit.amus},
                                   periodic_forcefield_kwargs=None,
                                   small_molecule_forcefield='gaff-2.11',
                                   nonperiodic_forcefield_kwargs={'nonbondedMethod': app.NoCutoff},
                                   molecules=None,
                                   cache=None)

modeller.addSolvent(system_generator.forcefield, model='tip3p', padding=9*unit.angstroms, ionicStrength=0.15*unit.molar)
topology = modeller.getTopology()
positions = modeller.getPositions()

# Canonicalize the solvated positions: turn tuples into np.array
positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
system = system_generator.create_system(topology)



In [25]:
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.annihilation.relative import RepartitionedHybridTopologyFactory
from perses.tests.utils import validate_endstate_energies
import random

aminos = ['ALA','ARG','ASN','ASP','CYS','GLN','GLU','GLY','HIS','ILE','LEU','LYS','MET','PHE','PRO','SER','THR','TRP','TYR','VAL']

ENERGY_THRESHOLD = 1e-6
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT

chain = 'C'
for res in topology.residues():
    if res.id == '2':
        wt_res = res.name
# aminos_updated = [amino for amino in aminos if amino not in [wt_res, 'PRO', 'HIS', 'TRP', 'PHE', 'TYR']]
# mutant = random.choice(aminos_updated)
mutant = "MET"
print(f'Making mutation {wt_res}->{mutant}')

# Create point mutation engine to mutate residue at id 2 to random amino acid
point_mutation_engine = PointMutationEngine(wildtype_topology=topology,
                                            system_generator=system_generator,
                                            chain_id=chain,
                                            max_point_mutants=1,
                                            residues_allowed_to_mutate=['2'],  # the residue ids allowed to mutate
                                            allowed_mutations=[('2', mutant)],
                                            aggregate=True)  # always allow aggregation

# Create topology proposal
topology_proposal = point_mutation_engine.propose(current_system=system, current_topology=topology)

# Create geometry engine
from perses.rjmc.geometry import FFAllAngleGeometryEngine
geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                           use_sterics=False,
                                           n_bond_divisions=100,
                                           n_angle_divisions=180,
                                           n_torsion_divisions=360,
                                           verbose=True,
                                           storage=None,
                                           bond_softening_constant=1.0,
                                           angle_softening_constant=1.0,
                                           neglect_angles=False,
                                           use_14_nonbondeds=True)

# Create geometry proposal
new_positions, logp_proposal = geometry_engine.propose(topology_proposal, positions, beta,
                                                               validate_energy_bookkeeping=True)
logp_reverse = geometry_engine.logp_reverse(topology_proposal, new_positions, positions, beta,
                                            validate_energy_bookkeeping=True)

if not topology_proposal.unique_new_atoms:
    assert geometry_engine.forward_final_context_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.forward_final_context_reduced_potential})"
    assert geometry_engine.forward_atoms_with_positions_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's forward atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.forward_atoms_with_positions_reduced_potential})"
    vacuum_added_valence_energy = 0.0
else:
    added_valence_energy = geometry_engine.forward_final_context_reduced_potential - geometry_engine.forward_atoms_with_positions_reduced_potential

if not topology_proposal.unique_old_atoms:
    assert geometry_engine.reverse_final_context_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.reverse_final_context_reduced_potential})"
    assert geometry_engine.reverse_atoms_with_positions_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.reverse_atoms_with_positions_reduced_potential})"
    subtracted_valence_energy = 0.0
else:
    subtracted_valence_energy = geometry_engine.reverse_final_context_reduced_potential - geometry_engine.reverse_atoms_with_positions_reduced_potential


2021-07-14 16:29:01,250:(10.63s):proposal_generator:	Conducting polymer point mutation proposal...


Making mutation ALA->MET


2021-07-14 16:29:02,102:(0.85s):proposal_generator:Using matching_criterion to chose best atom map
2021-07-14 16:29:02,110:(0.01s):proposal_generator:Scaffold has symmetry of 0
2021-07-14 16:29:02,113:(0.00s):proposal_generator:Two molecules are not similar to have a common scaffold
2021-07-14 16:29:02,116:(0.00s):proposal_generator:Proceeding with direct mapping of molecules, but please check atom mapping and the geometry of the ligands.
2021-07-14 16:29:02,121:(0.01s):proposal_generator:len [{13: 8}, {16: 9}, {15: 9}, {14: 9}, {13: 9}, {12: 9}, {11: 9}, {10: 9}, {16: 8}, {15: 8}, {14: 8}, {10: 7}, {12: 8}, {11: 8}, {10: 8}, {16: 7}, {15: 7}, {14: 7}, {13: 7}, {12: 7}, {11: 7}]
2021-07-14 16:29:02,124:(0.00s):proposal_generator:{13: 8}
2021-07-14 16:29:02,127:(0.00s):proposal_generator:{16: 9}
2021-07-14 16:29:02,129:(0.00s):proposal_generator:{15: 9}
2021-07-14 16:29:02,132:(0.00s):proposal_generator:{14: 9}
2021-07-14 16:29:02,134:(0.00s):proposal_generator:{13: 9}
2021-07-14 16:29:

conducting subsequent work with the following platform: CUDA


2021-07-14 16:29:07,898:(2.18s):geometry:setting atoms_with_positions context new positions


...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-07-14 16:29:27,985:(3.87s):geometry:total reduced potential before atom placement: 168.36662826283057


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-07-14 16:29:30,659:(2.67s):geometry:total reduced energy added from growth system: 19.11335785388209
2021-07-14 16:29:30,663:(0.00s):geometry:final reduced energy 187.47998570143525
2021-07-14 16:29:30,665:(0.00s):geometry:sum of energies: 187.47998611671267
2021-07-14 16:29:30,667:(0.00s):geometry:magnitude of difference in the energies: 4.1527740890501263e-07
2021-07-14 16:29:30,670:(0.00s):geometry:Final logp_proposal: -30443.871040691214


added energy components: [('CustomBondForce', 0.04562463240715887), ('CustomAngleForce', 0.11963264577965825), ('CustomTorsionForce', 7.457414545448549), ('CustomBondForce', 11.490686030246726)]


In [26]:
from perses.annihilation.relative import RxnHybridTopologyFactory
flatten_exceptions = True
flatten_torsions = True
endstate = None
scale_regions = None
htf = RxnHybridTopologyFactory(topology_proposal=topology_proposal,
                     current_positions=positions,
                     new_positions=new_positions,
                     use_dispersion_correction=False,
                     functions=None,
                     softcore_alpha=None,
                     bond_softening_constant=1.0,
                     angle_softening_constant=1.0,
                     soften_only_new=False,
                     neglected_new_angle_terms=[],
                     neglected_old_angle_terms=[],
                     softcore_LJ_v2=True,
                     softcore_electrostatics=True,
                     softcore_LJ_v2_alpha=0.85,
                     softcore_electrostatics_alpha=0.3,
                     softcore_sigma_Q=1.0,
                     interpolate_old_and_new_14s=flatten_exceptions,
                     omitted_terms=None,
                     flatten_torsions=flatten_torsions,
                     scale_regions=scale_regions)

2021-07-14 16:29:30,868:(0.20s):relative:*** Generating RxnHybridTopologyFactory ***
2021-07-14 16:29:30,871:(0.00s):relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
2021-07-14 16:29:31,124:(0.25s):relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
2021-07-14 16:29:31,127:(0.00s):relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
2021-07-14 16:29:31,129:(0.00s):relative:No unknown forces.
2021-07-14 16:29:31,131:(0.00s):relative:Nonbonded method to be used (i.e. from old system): 4
2021-07-14 16:29:31,134:(0.00s):relative:Adding and mapping old atoms to hybrid system...
2021-07-14 16:29:31,225:(0.09s):relative:Adding and mapping new atoms to hybrid system...
2021-07-14 16:29:31,228:(0.00s):relative:scale_templates: [['nonscale_lambda'], ['nonscale_region']

In [27]:
# ( nonscale_lambda_bonds * nonscale_region ) * (K/2)*(r-length)^2;

# K = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * K1 
# + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * K2;

# length = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * length1 
# + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * length2;


In [28]:
# Save htf
# with open("8mer_vacuum.pickle", "wb") as f:
#     pickle.dump(htf, f)
with open("8mer_solvent.pickle", "wb") as f:
    pickle.dump(htf, f)

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [29]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [30]:
htf.hybrid_system.getForces()

[<simtk.openmm.openmm.MonteCarloBarostat; proxy of <Swig Object of type 'OpenMM::MonteCarloBarostat *' at 0x2ae756524030> >,
 <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2ae7562a3d50> >,
 <simtk.openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2ae7562a3b40> >,
 <simtk.openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2ae7562a3e70> >,
 <simtk.openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2ae7562a3ea0> >,
 <simtk.openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2ae755d06900> >,
 <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2ae755d06f60> >,
 <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2ae755d06480> >]

In [31]:
custom_bond_force = htf.hybrid_system.getForce(3)

In [32]:
for i in htf._hybrid_to_environment_torsion_indices:
    p1, p2, p3, p4, hybrid_params = custom_bond_force.getTorsionParameters(i)
    print(p1, p2, p3, p4, hybrid_params)

2 1 0 3 (1.0, 1.0, 0.0, 1.0, 0.0, 3.3472000000000004, 1.0, 0.0, 0.0)
2 1 0 3 (1.0, 1.0, 0.0, 3.0, 3.141592653589793, 0.33472, 3.0, 3.141592653589793, 0.0)
2 1 0 4 (1.0, 1.0, 0.0, 1.0, 0.0, 3.3472000000000004, 1.0, 0.0, 0.0)
2 1 0 4 (1.0, 1.0, 0.0, 3.0, 3.141592653589793, 0.33472, 3.0, 3.141592653589793, 0.0)
2 1 0 5 (1.0, 1.0, 0.0, 1.0, 0.0, 3.3472000000000004, 1.0, 0.0, 0.0)
2 1 0 5 (1.0, 1.0, 0.0, 3.0, 3.141592653589793, 0.33472, 3.0, 3.141592653589793, 0.0)
43 44 45 62 (1.0, 1.0, 0.0, 3.0, 3.141592653589793, 2.3012, 3.0, 3.141592653589793, 0.0)
43 44 45 62 (1.0, 1.0, 0.0, 2.0, 3.141592653589793, 6.610720000000001, 2.0, 3.141592653589793, 0.0)
43 44 45 62 (1.0, 1.0, 0.0, 1.0, 3.141592653589793, 1.8828, 1.0, 3.141592653589793, 0.0)
43 44 47 48 (1.0, 1.0, 0.0, 4.0, 0.0, 0.305432, 4.0, 0.0, 0.0)
43 44 47 48 (1.0, 1.0, 0.0, 3.0, 0.0, 0.602496, 3.0, 0.0, 0.0)
43 44 47 48 (1.0, 1.0, 0.0, 2.0, 3.141592653589793, 1.0836560000000002, 2.0, 3.141592653589793, 0.0)
43 44 47 48 (1.0, 1.0, 0.0, 1.

In [33]:
test_bond_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 7.300105424153174), ('HarmonicAngleForce', 12.445685055931849), ('PeriodicTorsionForce', 156.24350906389375), ('NonbondedForce', -10344.197299426876), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.300105424153174), ('CustomAngleForce', 379.92447014121484), ('CustomTorsionForce', 156.24351836067592), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [34]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent.pickle", "rb") as f:
    htf = pickle.load(f)


In [35]:
test_bond_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 8.78035136547036), ('HarmonicAngleForce', 379.80483749543527), ('PeriodicTorsionForce', 167.34929278966283), ('NonbondedForce', -6815.4331543112885), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 8.780351365470361), ('CustomAngleForce', 379.92447014121484), ('CustomTorsionForce', 156.24351836067592), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


### Test one alchemical region, one scale region

In [36]:
import os
from pkg_resources import resource_filename
from perses.app import setup_relative_calculation
import pickle
from simtk import openmm, unit
from simtk.openmm import app
from openmmforcefields.generators import SystemGenerator
import numpy as np

In [37]:
# Test 8-mer peptide in solvent
peptide_filename = resource_filename('perses', 'data/8mer-example/4zuh_peptide_capped.pdb')
pdb = app.PDBFile(peptide_filename)
forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
# barostat = None
modeller = app.Modeller(pdb.topology, pdb.positions)
 
system_generator = SystemGenerator(forcefields=forcefield_files,
                                   barostat=barostat,
                                   forcefield_kwargs={'removeCMMotion': False,
                                                      'ewaldErrorTolerance': 0.00025,
                                                      'constraints': app.HBonds,
                                                      'hydrogenMass': 4 * unit.amus},
                                   periodic_forcefield_kwargs=None,
                                   small_molecule_forcefield='gaff-2.11',
                                   nonperiodic_forcefield_kwargs={'nonbondedMethod': app.NoCutoff},
                                   molecules=None,
                                   cache=None)

modeller.addSolvent(system_generator.forcefield, model='tip3p', padding=9*unit.angstroms, ionicStrength=0.15*unit.molar)
topology = modeller.getTopology()
positions = modeller.getPositions()

# Canonicalize the solvated positions: turn tuples into np.array
positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
system = system_generator.create_system(topology)



In [38]:
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.annihilation.relative import RepartitionedHybridTopologyFactory
from perses.tests.utils import validate_endstate_energies
import random

aminos = ['ALA','ARG','ASN','ASP','CYS','GLN','GLU','GLY','HIS','ILE','LEU','LYS','MET','PHE','PRO','SER','THR','TRP','TYR','VAL']

ENERGY_THRESHOLD = 1e-6
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT

chain = 'C'
for res in topology.residues():
    if res.id == '2':
        wt_res = res.name
aminos_updated = [amino for amino in aminos if amino not in [wt_res, 'PRO', 'HIS', 'TRP', 'PHE', 'TYR']]
mutant = random.choice(aminos_updated)
print(f'Making mutation {wt_res}->{mutant}')

# Create point mutation engine to mutate residue at id 2 to random amino acid
point_mutation_engine = PointMutationEngine(wildtype_topology=topology,
                                            system_generator=system_generator,
                                            chain_id=chain,
                                            max_point_mutants=1,
                                            residues_allowed_to_mutate=['2'],  # the residue ids allowed to mutate
                                            allowed_mutations=[('2', mutant)],
                                            aggregate=True)  # always allow aggregation

# Create topology proposal
topology_proposal = point_mutation_engine.propose(current_system=system, current_topology=topology)

# Create geometry engine
from perses.rjmc.geometry import FFAllAngleGeometryEngine
geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                           use_sterics=False,
                                           n_bond_divisions=100,
                                           n_angle_divisions=180,
                                           n_torsion_divisions=360,
                                           verbose=True,
                                           storage=None,
                                           bond_softening_constant=1.0,
                                           angle_softening_constant=1.0,
                                           neglect_angles=False,
                                           use_14_nonbondeds=True)

# Create geometry proposal
new_positions, logp_proposal = geometry_engine.propose(topology_proposal, positions, beta,
                                                               validate_energy_bookkeeping=True)
logp_reverse = geometry_engine.logp_reverse(topology_proposal, new_positions, positions, beta,
                                            validate_energy_bookkeeping=True)

if not topology_proposal.unique_new_atoms:
    assert geometry_engine.forward_final_context_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.forward_final_context_reduced_potential})"
    assert geometry_engine.forward_atoms_with_positions_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's forward atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.forward_atoms_with_positions_reduced_potential})"
    vacuum_added_valence_energy = 0.0
else:
    added_valence_energy = geometry_engine.forward_final_context_reduced_potential - geometry_engine.forward_atoms_with_positions_reduced_potential

if not topology_proposal.unique_old_atoms:
    assert geometry_engine.reverse_final_context_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.reverse_final_context_reduced_potential})"
    assert geometry_engine.reverse_atoms_with_positions_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.reverse_atoms_with_positions_reduced_potential})"
    subtracted_valence_energy = 0.0
else:
    subtracted_valence_energy = geometry_engine.reverse_final_context_reduced_potential - geometry_engine.reverse_atoms_with_positions_reduced_potential


2021-07-14 16:29:56,970:(25.28s):proposal_generator:	Conducting polymer point mutation proposal...


Making mutation ALA->CYS


2021-07-14 16:29:57,574:(0.60s):proposal_generator:Using matching_criterion to chose best atom map
2021-07-14 16:29:57,579:(0.00s):proposal_generator:Scaffold has symmetry of 0
2021-07-14 16:29:57,581:(0.00s):proposal_generator:Two molecules are not similar to have a common scaffold
2021-07-14 16:29:57,584:(0.00s):proposal_generator:Proceeding with direct mapping of molecules, but please check atom mapping and the geometry of the ligands.
2021-07-14 16:29:57,587:(0.00s):proposal_generator:len [{8: 7}, {9: 7}, {10: 7}, {8: 8}, {9: 8}, {10: 8}, {8: 9}, {9: 9}, {10: 9}]
2021-07-14 16:29:57,590:(0.00s):proposal_generator:{8: 7}
2021-07-14 16:29:57,592:(0.00s):proposal_generator:{9: 7}
2021-07-14 16:29:57,594:(0.00s):proposal_generator:{10: 7}
2021-07-14 16:29:57,597:(0.00s):proposal_generator:{8: 8}
2021-07-14 16:29:57,599:(0.00s):proposal_generator:{9: 8}
2021-07-14 16:29:57,601:(0.00s):proposal_generator:{10: 8}
2021-07-14 16:29:57,604:(0.00s):proposal_generator:{8: 9}
2021-07-14 16:29:5

conducting subsequent work with the following platform: CUDA


2021-07-14 16:30:03,508:(2.31s):geometry:setting atoms_with_positions context new positions


...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-07-14 16:30:18,746:(1.43s):geometry:total reduced potential before atom placement: 168.36662826283057


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-07-14 16:30:20,391:(1.64s):geometry:total reduced energy added from growth system: 19.11335785388209
2021-07-14 16:30:20,394:(0.00s):geometry:final reduced energy 187.47998570143525
2021-07-14 16:30:20,396:(0.00s):geometry:sum of energies: 187.47998611671267
2021-07-14 16:30:20,399:(0.00s):geometry:magnitude of difference in the energies: 4.1527740890501263e-07
2021-07-14 16:30:20,401:(0.00s):geometry:Final logp_proposal: -30364.724484340997


added energy components: [('CustomBondForce', 0.04562463240715887), ('CustomAngleForce', 0.11963264577965825), ('CustomTorsionForce', 7.457414545448549), ('CustomBondForce', 11.490686030246726)]


In [39]:
from perses.annihilation.relative import RxnHybridTopologyFactory
flatten_exceptions = True
flatten_torsions = True
endstate = None
scale_regions = [list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])]
htf = RxnHybridTopologyFactory(topology_proposal=topology_proposal,
                     current_positions=positions,
                     new_positions=new_positions,
                     use_dispersion_correction=False,
                     functions=None,
                     softcore_alpha=None,
                     bond_softening_constant=1.0,
                     angle_softening_constant=1.0,
                     soften_only_new=False,
                     neglected_new_angle_terms=[],
                     neglected_old_angle_terms=[],
                     softcore_LJ_v2=True,
                     softcore_electrostatics=True,
                     softcore_LJ_v2_alpha=0.85,
                     softcore_electrostatics_alpha=0.3,
                     softcore_sigma_Q=1.0,
                     interpolate_old_and_new_14s=flatten_exceptions,
                     omitted_terms=None,
                     flatten_torsions=flatten_torsions,
                     scale_regions=scale_regions)

2021-07-14 16:30:20,584:(0.18s):relative:*** Generating RxnHybridTopologyFactory ***
2021-07-14 16:30:20,587:(0.00s):relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
2021-07-14 16:30:20,848:(0.26s):relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
2021-07-14 16:30:20,851:(0.00s):relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
2021-07-14 16:30:20,853:(0.00s):relative:No unknown forces.
2021-07-14 16:30:20,856:(0.00s):relative:Nonbonded method to be used (i.e. from old system): 4
2021-07-14 16:30:20,858:(0.00s):relative:Adding and mapping old atoms to hybrid system...
2021-07-14 16:30:20,950:(0.09s):relative:Adding and mapping new atoms to hybrid system...
2021-07-14 16:30:20,953:(0.00s):relative:scale_templates: [['nonscale_lambda', 'scale_lambda_0', 'i

In [40]:
# Save htf
with open("8mer_solvent_scale_region.pickle", "wb") as f:
    pickle.dump(htf, f)

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [41]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [42]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[24, 25, 26, 21, 5575, 5576, 5577, 5578, 5579]

In [43]:
test_bond_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 7.300105424153174), ('HarmonicAngleForce', 12.445685055931849), ('PeriodicTorsionForce', 156.24350906389375), ('NonbondedForce', -10441.051660671816), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.300105424153174), ('CustomAngleForce', 220.6742853389168), ('CustomTorsionForce', 156.24351836067592), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [44]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [45]:
test_bond_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 12.5351234911881), ('HarmonicAngleForce', 220.55465269313717), ('PeriodicTorsionForce', 164.43732525160556), ('NonbondedForce', 80043.38328338433), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 12.5351234911881), ('CustomAngleForce', 220.6742853389168), ('CustomTorsionForce', 156.24351836067592), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


## Barnase:barstar (old htf, no naked charge fix or counterion fix)

### Test one alchemical region, no scale regions

In [48]:
from perses.app.relative_point_mutation_setup import PointMutationExecutor
from openeye import oechem

In [50]:
from __future__ import absolute_import

from perses.utils.openeye import createOEMolFromSDF, extractPositionsFromOEMol
from perses.annihilation.relative import HybridTopologyFactory, RepartitionedHybridTopologyFactory, RxnHybridTopologyFactory
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.rjmc.geometry import FFAllAngleGeometryEngine

import simtk.openmm as openmm
import simtk.openmm.app as app
import simtk.unit as unit
import numpy as np
from openmoltools import forcefield_generators
import mdtraj as md
from openmmtools.constants import kB
from perses.tests.utils import validate_endstate_energies
from openff.toolkit.topology import Molecule
from openmmforcefields.generators import SystemGenerator

ENERGY_THRESHOLD = 1e-2
temperature = 300 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
ring_amino_acids = ['TYR', 'PHE', 'TRP', 'PRO', 'HIS']

# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

class PointMutationExecutor2(PointMutationExecutor):
    """
    Simple, stripped-down class to create a protein-ligand system and allow a mutation of a protein.
    this will allow support for the creation of _two_ relative free energy calculations:
        1. 'wildtype' - 'point mutant' complex hybrid.
        2. 'wildtype' - 'point mutant' protein hybrid (i.e. with ligand of interest unbound)
    Example (create full point mutation executor and run parallel tempering on both complex and apo phases):
        from pkg_resources import resource_filename
        protein_path = 'data/perses_jacs_systems/thrombin/Thrombin_protein.pdb'
        ligands_path = 'data/perses_jacs_systems/thrombin/Thrombin_ligands.sdf'
        protein_filename = resource_filename('openmmforcefields', protein_path)
        ligand_input = resource_filename('openmmforcefields', ligands_path)
        pm_delivery = PointMutationExecutor(protein_filename=protein_filename,
                                    mutation_chain_id='2',
                                    mutation_residue_id='198',
                                     proposed_residue='THR',
                                     phase='complex',
                                     conduct_endstate_validation=False,
                                     ligand_input=ligand_input,
                                     ligand_index=0,
                                     forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'],
                                     barostat=openmm.MonteCarloBarostat(1.0 * unit.atmosphere, temperature, 50),
                                     forcefield_kwargs={'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'nonbondedMethod': app.PME, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus},
                                     small_molecule_forcefields='gaff-2.11')
        complex_htf = pm_delivery.get_complex_htf()
        apo_htf = pm_delivery.get_apo_htf()
        # Now we can build the hybrid repex samplers
        from perses.annihilation.lambda_protocol import LambdaProtocol
        from openmmtools.multistate import MultiStateReporter
        from perses.samplers.multistate import HybridRepexSampler
        from openmmtools import mcmc
        suffix = 'run'; selection = 'not water'; checkpoint_interval = 10; n_states = 11; n_cycles = 5000
        for htf in [complex_htf, apo_htf]:
            lambda_protocol = LambdaProtocol(functions='default')
            reporter_file = 'reporter.nc'
            reporter = MultiStateReporter(reporter_file, analysis_particle_indices = htf.hybrid_topology.select(selection), checkpoint_interval = checkpoint_interval)
            hss = HybridRepexSampler(mcmc_moves=mcmc.LangevinSplittingDynamicsMove(timestep= 4.0 * unit.femtoseconds,
                                                                                  collision_rate=5.0 / unit.picosecond,
                                                                                  n_steps=250,
                                                                                  reassign_velocities=False,
                                                                                  n_restart_attempts=20,
                                                                                  splitting="V R R R O R R R V",
                                                                                  constraint_tolerance=1e-06),
                                                                                  hybrid_factory=htf, online_analysis_interval=10)
            hss.setup(n_states=n_states, temperature=300*unit.kelvin, storage_file=reporter, lambda_protocol=lambda_protocol, endstates=False)
            hss.extend(n_cycles)
    """
    def __init__(self,
                 protein_filename,
                 mutation_chain_id,
                 mutation_residue_id,
                 proposed_residue,
                 phase='complex',
                 conduct_endstate_validation=True,
                 ligand_input=None,
                 ligand_index=0,
                 water_model='tip3p',
                 ionic_strength=0.15 * unit.molar,
                 forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'],
                 barostat=openmm.MonteCarloBarostat(1.0 * unit.atmosphere, temperature, 50),
                 forcefield_kwargs={'removeCMMotion': False, 'ewaldErrorTolerance': 0.00025, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus},
                 periodic_forcefield_kwargs={'nonbondedMethod': app.PME},
                 nonperiodic_forcefield_kwargs=None,
                 small_molecule_forcefields='gaff-2.11',
                 complex_box_dimensions=None,
                 apo_box_dimensions=None,
                 flatten_torsions=False,
                 flatten_exceptions=False,
                 repartitioned_endstate=None,
                 rxn_field = True,
                 **kwargs):
        """
        arguments
            protein_filename : str
                path to protein (to mutate); .pdb
            mutation_chain_id : str
                name of the chain to be mutated
            mutation_residue_id : str
                residue id to change
            proposed_residue : str
                three letter code of the residue to mutate to
            phase : str, default complex
                if phase == vacuum, then the complex will not be solvated with water; else, it will be solvated with tip3p
            conduct_endstate_validation : bool, default True
                whether to conduct an endstate validation of the HybridTopologyFactory. If using the RepartitionedHybridTopologyFactory,
                endstate validation cannot and will not be conducted.
            ligand_file : str, default None
                path to ligand of interest (i.e. small molecule or protein); .sdf or .pdb
            ligand_index : int, default 0
                which ligand to use
            water_model : str, default 'tip3p'
                solvent model to use for solvation
            ionic_strength : float * unit.molar, default 0.15 * unit.molar
                the total concentration of ions (both positive and negative) to add using Modeller.
                This does not include ions that are added to neutralize the system.
                Note that only monovalent ions are currently supported.
            forcefield_files : list of str, default ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
                forcefield files for proteins and solvent
            barostat : openmm.MonteCarloBarostat, default openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
                barostat to use
            forcefield_kwargs : dict, default {'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus}
                forcefield kwargs for system parametrization
            periodic_forcefield_kwargs : dict, default {'nonbondedMethod': app.PME}
                periodic forcefield kwargs for system parametrization
            nonperiodic_forcefield_kwargs : dict, default None
                non-periodic forcefield kwargs for system parametrization
            small_molecule_forcefields : str, default 'gaff-2.11'
                the forcefield string for small molecule parametrization
            complex_box_dimensions : Vec3, default None
                define box dimensions of complex phase;
                if None, padding is 1nm
            apo_box_dimensions :  Vec3, default None
                define box dimensions of apo phase phase;
                if None, padding is 1nm
            flatten_torsions : bool, default False
                in the htf, flatten torsions involving unique new atoms at lambda = 0 and unique old atoms are lambda = 1
            flatten_exceptions : bool, default False
                in the htf, flatten exceptions involving unique new atoms at lambda = 0 and unique old atoms at lambda = 1
            repartitioned_endstate : int, default None
                the endstate (0 or 1) at which to build the RepartitionedHybridTopologyFactory. By default, this is None,
                meaning a vanilla HybridTopologyFactory will be built.
        TODO : allow argument for spectator ligands besides the 'ligand_file'
        """

        # First thing to do is load the apo protein to mutate...
        protein_pdbfile = open(protein_filename, 'r')
        protein_pdb = app.PDBFile(protein_pdbfile)
        protein_pdbfile.close()
        protein_positions, protein_topology, protein_md_topology = protein_pdb.positions, protein_pdb.topology, md.Topology.from_openmm(protein_pdb.topology)
        protein_topology = protein_md_topology.to_openmm()
        protein_n_atoms = protein_md_topology.n_atoms

        # Load the ligand, if present
        molecules = []
        if ligand_input:
            if isinstance(ligand_input, str):
                if ligand_input.endswith('.sdf'): # small molecule
                        ligand_mol = createOEMolFromSDF(ligand_input, index=ligand_index)
                        molecules.append(Molecule.from_openeye(ligand_mol, allow_undefined_stereo=False))
                        ligand_positions, ligand_topology = extractPositionsFromOEMol(ligand_mol),  forcefield_generators.generateTopologyFromOEMol(ligand_mol)
                        ligand_md_topology = md.Topology.from_openmm(ligand_topology)
                        ligand_n_atoms = ligand_md_topology.n_atoms

                if ligand_input.endswith('pdb'): # protein
                    ligand_pdbfile = open(ligand_input, 'r')
                    ligand_pdb = app.PDBFile(ligand_pdbfile)
                    ligand_pdbfile.close()
                    ligand_positions, ligand_topology, ligand_md_topology = ligand_pdb.positions, ligand_pdb.topology, md.Topology.from_openmm(
                        ligand_pdb.topology)
                    ligand_n_atoms = ligand_md_topology.n_atoms

            elif isinstance(ligand_input, oechem.OEMol): # oemol object
                molecules.append(Molecule.from_openeye(ligand_input, allow_undefined_stereo=False))
                ligand_positions, ligand_topology = extractPositionsFromOEMol(ligand_input),  forcefield_generators.generateTopologyFromOEMol(ligand_input)
                ligand_md_topology = md.Topology.from_openmm(ligand_topology)
                ligand_n_atoms = ligand_md_topology.n_atoms

            else:
                _logger.warning(f'ligand filetype not recognised. Please provide a path to a .pdb or .sdf file')
                return

            # Now create a complex
            complex_md_topology = protein_md_topology.join(ligand_md_topology)
            complex_topology = complex_md_topology.to_openmm()
            complex_positions = unit.Quantity(np.zeros([protein_n_atoms + ligand_n_atoms, 3]), unit=unit.nanometers)
            complex_positions[:protein_n_atoms, :] = protein_positions
            complex_positions[protein_n_atoms:, :] = ligand_positions

        # Now for a system_generator
        self.system_generator = SystemGenerator(forcefields=forcefield_files,
                                                barostat=barostat,
                                                forcefield_kwargs=forcefield_kwargs,
                                                periodic_forcefield_kwargs=periodic_forcefield_kwargs,
                                                nonperiodic_forcefield_kwargs=nonperiodic_forcefield_kwargs,
                                                small_molecule_forcefield=small_molecule_forcefields,
                                                molecules=molecules,
                                                cache=None)

        # Solvate apo and complex...
        apo_input = list(self._solvate(protein_topology, protein_positions, water_model, phase, ionic_strength, apo_box_dimensions))
        inputs = [apo_input]
        if ligand_input:
            inputs.append(self._solvate(complex_topology, complex_positions, water_model, phase, ionic_strength, complex_box_dimensions))

        geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                                use_sterics=False,
                                                n_bond_divisions=100,
                                                n_angle_divisions=180,
                                                n_torsion_divisions=360,
                                                verbose=True,
                                                storage=None,
                                                bond_softening_constant=1.0,
                                                angle_softening_constant=1.0,
                                                neglect_angles = False,
                                                use_14_nonbondeds = True)


        # Run pipeline...
        htfs = []
        for (top, pos, sys) in inputs:
            point_mutation_engine = PointMutationEngine(wildtype_topology=top,
                                                                 system_generator=self.system_generator,
                                                                 chain_id=mutation_chain_id, # Denote the chain id allowed to mutate (it's always a string variable)
                                                                 max_point_mutants=1,
                                                                 residues_allowed_to_mutate=[mutation_residue_id], # The residue ids allowed to mutate
                                                                 allowed_mutations=[(mutation_residue_id, proposed_residue)], # The residue ids allowed to mutate with the three-letter code allowed to change
                                                                 aggregate=True) # Always allow aggregation

            topology_proposal = point_mutation_engine.propose(sys, top)

            # Only validate energy bookkeeping if the WT and proposed residues do not involve rings
            old_res = [res for res in top.residues() if res.id == mutation_residue_id][0]
            validate_bool = False if old_res.name in ring_amino_acids or proposed_residue in ring_amino_acids else True
            new_positions, logp_proposal = geometry_engine.propose(topology_proposal, pos, beta,
                                                                   validate_energy_bookkeeping=validate_bool)
            logp_reverse = geometry_engine.logp_reverse(topology_proposal, new_positions, pos, beta,
                                                        validate_energy_bookkeeping=validate_bool)

            if rxn_field:
                factory = RxnHybridTopologyFactory
            else:
                if repartitioned_endstate is None:
                    factory = HybridTopologyFactory
                elif repartitioned_endstate in [0, 1]:
                    factory = RepartitionedHybridTopologyFactory

            forward_htf = factory(topology_proposal=topology_proposal,
                                  current_positions=pos,
                                  new_positions=new_positions,
                                  use_dispersion_correction=False,
                                  functions=None,
                                  softcore_alpha=None,
                                  bond_softening_constant=1.0,
                                  angle_softening_constant=1.0,
                                  soften_only_new=False,
                                  neglected_new_angle_terms=[],
                                  neglected_old_angle_terms=[],
                                  softcore_LJ_v2=True,
                                  softcore_electrostatics=True,
                                  softcore_LJ_v2_alpha=0.85,
                                  softcore_electrostatics_alpha=0.3,
                                  softcore_sigma_Q=1.0,
                                  interpolate_old_and_new_14s=flatten_exceptions,
                                  omitted_terms=None,
                                  endstate=repartitioned_endstate,
                                  flatten_torsions=flatten_torsions)

            if not topology_proposal.unique_new_atoms:
                assert geometry_engine.forward_final_context_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.forward_final_context_reduced_potential})"
                assert geometry_engine.forward_atoms_with_positions_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's forward atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.forward_atoms_with_positions_reduced_potential})"
            else:
                added_valence_energy = geometry_engine.forward_final_context_reduced_potential - geometry_engine.forward_atoms_with_positions_reduced_potential

            if not topology_proposal.unique_old_atoms:
                assert geometry_engine.reverse_final_context_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.reverse_final_context_reduced_potential})"
                assert geometry_engine.reverse_atoms_with_positions_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.reverse_atoms_with_positions_reduced_potential})"
                subtracted_valence_energy = 0.0
            else:
                subtracted_valence_energy = geometry_engine.reverse_final_context_reduced_potential - geometry_engine.reverse_atoms_with_positions_reduced_potential


            if conduct_endstate_validation and repartitioned_endstate is None:
                zero_state_error, one_state_error = validate_endstate_energies(forward_htf._topology_proposal, forward_htf, added_valence_energy, subtracted_valence_energy, beta=beta, ENERGY_THRESHOLD=ENERGY_THRESHOLD)
                if zero_state_error > ENERGY_THRESHOLD:
                    _logger.warning(f"Reduced potential difference of the nonalchemical and alchemical Lambda = 0 state is above the threshold ({ENERGY_THRESHOLD}): {zero_state_error}")
                if one_state_error > ENERGY_THRESHOLD:
                    _logger.warning(f"Reduced potential difference of the nonalchemical and alchemical Lambda = 1 state is above the threshold ({ENERGY_THRESHOLD}): {one_state_error}")
            else:
                pass

            htfs.append(forward_htf)

        self.apo_htf = htfs[0]
        self.complex_htf = htfs[1] if ligand_input else None

In [51]:
solvent_delivery = PointMutationExecutor2("../../input/mmc2_barstar.pdb",
                        '1',
                        "42",
                        "ALA",
                        ligand_input="../../input/mmc2_barnase.pdb",
                        ionic_strength=0.05*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True, 
                        conduct_endstate_validation=False,
                        rxn_field=True
                       )

2021-07-14 16:31:30,566:(69.23s):setup:solvating at 0.05 M using tip3p
2021-07-14 16:31:36,622:(6.06s):setup:solvating at 0.05 M using tip3p
2021-07-14 16:31:48,288:(11.67s):proposal_generator:	Conducting polymer point mutation proposal...
2021-07-14 16:31:51,919:(3.63s):proposal_generator:Using matching_criterion to chose best atom map
2021-07-14 16:31:51,925:(0.01s):proposal_generator:Scaffold has symmetry of 0
2021-07-14 16:31:51,928:(0.00s):proposal_generator:Two molecules are not similar to have a common scaffold
2021-07-14 16:31:51,931:(0.00s):proposal_generator:Proceeding with direct mapping of molecules, but please check atom mapping and the geometry of the ligands.
2021-07-14 16:31:51,935:(0.00s):proposal_generator:len [{7: 9}, {8: 9}, {9: 9}, {7: 10}, {8: 10}, {9: 10}, {7: 11}, {8: 11}, {9: 11}, {7: 12}, {8: 12}, {9: 12}, {7: 13}, {8: 13}, {9: 13}]
2021-07-14 16:31:51,937:(0.00s):proposal_generator:{7: 9}
2021-07-14 16:31:51,940:(0.00s):proposal_generator:{8: 9}
2021-07-14 16

conducting subsequent work with the following platform: CUDA


2021-07-14 16:32:09,122:(2.94s):geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


2021-07-14 16:32:10,457:(1.33s):geometry:There are 4 new atoms
2021-07-14 16:32:10,465:(0.01s):geometry:	reduced angle potential = 0.18086335368450154.
2021-07-14 16:32:10,746:(0.28s):geometry:	reduced angle potential = 0.039149204817402854.
2021-07-14 16:32:11,024:(0.28s):geometry:	reduced angle potential = 0.8663377102209178.
2021-07-14 16:32:11,301:(0.28s):geometry:	reduced angle potential = 0.08942060250388176.
2021-07-14 16:32:12,021:(0.72s):geometry:	beginning construction of no_nonbonded final system...
2021-07-14 16:32:12,025:(0.00s):geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
2021-07-14 16:32:13,771:(1.75s):geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
2021-07-14 16:32:13,774:(0.00s):geometry:	there are 733 bond forces in the no-nonbonded final system
2021-07-14 16:

...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-07-14 16:35:24,281:(3.41s):geometry:total reduced energy added from growth system: -62.3900963283472
2021-07-14 16:35:24,285:(0.00s):geometry:final reduced energy 4730.077774354767
2021-07-14 16:35:24,288:(0.00s):geometry:sum of energies: 4730.07777434895
2021-07-14 16:35:24,290:(0.00s):geometry:magnitude of difference in the energies: 5.817554438181105e-09
2021-07-14 16:35:24,292:(0.00s):geometry:Final logp_proposal: -10169.46087721557


added energy components: [('CustomBondForce', 0.4083956553523612), ('CustomAngleForce', 1.622079514297996), ('CustomTorsionForce', 17.503137529087592), ('CustomBondForce', -81.92370902708515)]


2021-07-14 16:35:24,456:(0.16s):relative:*** Generating RxnHybridTopologyFactory ***
2021-07-14 16:35:24,459:(0.00s):relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
2021-07-14 16:35:26,417:(1.96s):relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
2021-07-14 16:35:26,422:(0.00s):relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
2021-07-14 16:35:26,424:(0.00s):relative:No unknown forces.
2021-07-14 16:35:26,427:(0.00s):relative:Nonbonded method to be used (i.e. from old system): 4
2021-07-14 16:35:26,429:(0.00s):relative:Adding and mapping old atoms to hybrid system...
2021-07-14 16:35:26,904:(0.47s):relative:Adding and mapping new atoms to hybrid system...
2021-07-14 16:35:26,907:(0.00s):relative:scale_templates: [['nonscale_lambda'], ['nonscale_region']

2021-07-14 16:35:34,085:(0.01s):relative:hybrid_index_pair (2243, 2242, 2244, 2253) was not found in the new_term_collector, but (2242, 2243, 2244, 2253) has the same atoms and terms, so (2242, 2243, 2244, 2253) will be removed from the new term collector
2021-07-14 16:35:34,096:(0.01s):relative:hybrid_index_pair (2270, 2277, 2272, 2278) was not found in the new_term_collector, but (2270, 2278, 2272, 2277) has the same atoms and terms, so (2270, 2278, 2272, 2277) will be removed from the new term collector
2021-07-14 16:35:34,107:(0.01s):relative:hybrid_index_pair (2286, 2288, 2287, 2289) was not found in the new_term_collector, but (2288, 2287, 2286, 2289) has the same atoms and terms, so (2288, 2287, 2286, 2289) will be removed from the new term collector
2021-07-14 16:35:34,120:(0.01s):relative:hybrid_index_pair (2436, 2435, 2434, 2437) was not found in the new_term_collector, but (2434, 2436, 2435, 2437) has the same atoms and terms, so (2434, 2436, 2435, 2437) will be removed from

In [52]:
pickle.dump(solvent_delivery.get_apo_htf(), open("T42A_apo.pickle", "wb" ))
pickle.dump(solvent_delivery.get_complex_htf(), open("T42A_complex.pickle", "wb" ))


#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [53]:
# Load htf
with open("T42A_complex.pickle", "rb") as f:
    htf = pickle.load(f)

In [54]:
test_bond_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 189.69678123582867), ('HarmonicAngleForce', 791.7783222646883), ('PeriodicTorsionForce', 3830.526380065008), ('NonbondedForce', -23340.322141169163), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 189.69678123582867), ('CustomAngleForce', 1005.3622570356476), ('CustomTorsionForce', 3830.526548711567), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [55]:
# Load htf
with open("T42A_complex.pickle", "rb") as f:
    htf = pickle.load(f)

In [56]:
test_bond_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 190.24081990761337), ('HarmonicAngleForce', 1003.7401775213498), ('PeriodicTorsionForce', 3824.0198045870984), ('NonbondedForce', -22620.866336776005), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 190.24081990761337), ('CustomAngleForce', 1005.3622570356476), ('CustomTorsionForce', 3830.526548711567), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


In [57]:
scale_region = list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

### Test one alchemical region, no scale regions

In [58]:
from perses.app.relative_point_mutation_setup import PointMutationExecutor

In [60]:
from __future__ import absolute_import

from perses.utils.openeye import createOEMolFromSDF, extractPositionsFromOEMol
from perses.annihilation.relative import HybridTopologyFactory, RepartitionedHybridTopologyFactory, RxnHybridTopologyFactory
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.rjmc.geometry import FFAllAngleGeometryEngine

import simtk.openmm as openmm
import simtk.openmm.app as app
import simtk.unit as unit
import numpy as np
from openmoltools import forcefield_generators
import mdtraj as md
from openmmtools.constants import kB
from perses.tests.utils import validate_endstate_energies
from openff.toolkit.topology import Molecule
from openmmforcefields.generators import SystemGenerator

ENERGY_THRESHOLD = 1e-2
temperature = 300 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
ring_amino_acids = ['TYR', 'PHE', 'TRP', 'PRO', 'HIS']

# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

class PointMutationExecutor2(PointMutationExecutor):
    """
    Simple, stripped-down class to create a protein-ligand system and allow a mutation of a protein.
    this will allow support for the creation of _two_ relative free energy calculations:
        1. 'wildtype' - 'point mutant' complex hybrid.
        2. 'wildtype' - 'point mutant' protein hybrid (i.e. with ligand of interest unbound)
    Example (create full point mutation executor and run parallel tempering on both complex and apo phases):
        from pkg_resources import resource_filename
        protein_path = 'data/perses_jacs_systems/thrombin/Thrombin_protein.pdb'
        ligands_path = 'data/perses_jacs_systems/thrombin/Thrombin_ligands.sdf'
        protein_filename = resource_filename('openmmforcefields', protein_path)
        ligand_input = resource_filename('openmmforcefields', ligands_path)
        pm_delivery = PointMutationExecutor(protein_filename=protein_filename,
                                    mutation_chain_id='2',
                                    mutation_residue_id='198',
                                     proposed_residue='THR',
                                     phase='complex',
                                     conduct_endstate_validation=False,
                                     ligand_input=ligand_input,
                                     ligand_index=0,
                                     forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'],
                                     barostat=openmm.MonteCarloBarostat(1.0 * unit.atmosphere, temperature, 50),
                                     forcefield_kwargs={'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'nonbondedMethod': app.PME, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus},
                                     small_molecule_forcefields='gaff-2.11')
        complex_htf = pm_delivery.get_complex_htf()
        apo_htf = pm_delivery.get_apo_htf()
        # Now we can build the hybrid repex samplers
        from perses.annihilation.lambda_protocol import LambdaProtocol
        from openmmtools.multistate import MultiStateReporter
        from perses.samplers.multistate import HybridRepexSampler
        from openmmtools import mcmc
        suffix = 'run'; selection = 'not water'; checkpoint_interval = 10; n_states = 11; n_cycles = 5000
        for htf in [complex_htf, apo_htf]:
            lambda_protocol = LambdaProtocol(functions='default')
            reporter_file = 'reporter.nc'
            reporter = MultiStateReporter(reporter_file, analysis_particle_indices = htf.hybrid_topology.select(selection), checkpoint_interval = checkpoint_interval)
            hss = HybridRepexSampler(mcmc_moves=mcmc.LangevinSplittingDynamicsMove(timestep= 4.0 * unit.femtoseconds,
                                                                                  collision_rate=5.0 / unit.picosecond,
                                                                                  n_steps=250,
                                                                                  reassign_velocities=False,
                                                                                  n_restart_attempts=20,
                                                                                  splitting="V R R R O R R R V",
                                                                                  constraint_tolerance=1e-06),
                                                                                  hybrid_factory=htf, online_analysis_interval=10)
            hss.setup(n_states=n_states, temperature=300*unit.kelvin, storage_file=reporter, lambda_protocol=lambda_protocol, endstates=False)
            hss.extend(n_cycles)
    """
    def __init__(self,
                 protein_filename,
                 mutation_chain_id,
                 mutation_residue_id,
                 proposed_residue,
                 phase='complex',
                 conduct_endstate_validation=True,
                 ligand_input=None,
                 ligand_index=0,
                 water_model='tip3p',
                 ionic_strength=0.15 * unit.molar,
                 forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'],
                 barostat=openmm.MonteCarloBarostat(1.0 * unit.atmosphere, temperature, 50),
                 forcefield_kwargs={'removeCMMotion': False, 'ewaldErrorTolerance': 0.00025, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus},
                 periodic_forcefield_kwargs={'nonbondedMethod': app.PME},
                 nonperiodic_forcefield_kwargs=None,
                 small_molecule_forcefields='gaff-2.11',
                 complex_box_dimensions=None,
                 apo_box_dimensions=None,
                 flatten_torsions=False,
                 flatten_exceptions=False,
                 repartitioned_endstate=None,
                 rxn_field = True,
                 **kwargs):
        """
        arguments
            protein_filename : str
                path to protein (to mutate); .pdb
            mutation_chain_id : str
                name of the chain to be mutated
            mutation_residue_id : str
                residue id to change
            proposed_residue : str
                three letter code of the residue to mutate to
            phase : str, default complex
                if phase == vacuum, then the complex will not be solvated with water; else, it will be solvated with tip3p
            conduct_endstate_validation : bool, default True
                whether to conduct an endstate validation of the HybridTopologyFactory. If using the RepartitionedHybridTopologyFactory,
                endstate validation cannot and will not be conducted.
            ligand_file : str, default None
                path to ligand of interest (i.e. small molecule or protein); .sdf or .pdb
            ligand_index : int, default 0
                which ligand to use
            water_model : str, default 'tip3p'
                solvent model to use for solvation
            ionic_strength : float * unit.molar, default 0.15 * unit.molar
                the total concentration of ions (both positive and negative) to add using Modeller.
                This does not include ions that are added to neutralize the system.
                Note that only monovalent ions are currently supported.
            forcefield_files : list of str, default ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
                forcefield files for proteins and solvent
            barostat : openmm.MonteCarloBarostat, default openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
                barostat to use
            forcefield_kwargs : dict, default {'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus}
                forcefield kwargs for system parametrization
            periodic_forcefield_kwargs : dict, default {'nonbondedMethod': app.PME}
                periodic forcefield kwargs for system parametrization
            nonperiodic_forcefield_kwargs : dict, default None
                non-periodic forcefield kwargs for system parametrization
            small_molecule_forcefields : str, default 'gaff-2.11'
                the forcefield string for small molecule parametrization
            complex_box_dimensions : Vec3, default None
                define box dimensions of complex phase;
                if None, padding is 1nm
            apo_box_dimensions :  Vec3, default None
                define box dimensions of apo phase phase;
                if None, padding is 1nm
            flatten_torsions : bool, default False
                in the htf, flatten torsions involving unique new atoms at lambda = 0 and unique old atoms are lambda = 1
            flatten_exceptions : bool, default False
                in the htf, flatten exceptions involving unique new atoms at lambda = 0 and unique old atoms at lambda = 1
            repartitioned_endstate : int, default None
                the endstate (0 or 1) at which to build the RepartitionedHybridTopologyFactory. By default, this is None,
                meaning a vanilla HybridTopologyFactory will be built.
        TODO : allow argument for spectator ligands besides the 'ligand_file'
        """

        # First thing to do is load the apo protein to mutate...
        protein_pdbfile = open(protein_filename, 'r')
        protein_pdb = app.PDBFile(protein_pdbfile)
        protein_pdbfile.close()
        protein_positions, protein_topology, protein_md_topology = protein_pdb.positions, protein_pdb.topology, md.Topology.from_openmm(protein_pdb.topology)
        protein_topology = protein_md_topology.to_openmm()
        protein_n_atoms = protein_md_topology.n_atoms

        # Load the ligand, if present
        molecules = []
        if ligand_input:
            if isinstance(ligand_input, str):
                if ligand_input.endswith('.sdf'): # small molecule
                        ligand_mol = createOEMolFromSDF(ligand_input, index=ligand_index)
                        molecules.append(Molecule.from_openeye(ligand_mol, allow_undefined_stereo=False))
                        ligand_positions, ligand_topology = extractPositionsFromOEMol(ligand_mol),  forcefield_generators.generateTopologyFromOEMol(ligand_mol)
                        ligand_md_topology = md.Topology.from_openmm(ligand_topology)
                        ligand_n_atoms = ligand_md_topology.n_atoms

                if ligand_input.endswith('pdb'): # protein
                    ligand_pdbfile = open(ligand_input, 'r')
                    ligand_pdb = app.PDBFile(ligand_pdbfile)
                    ligand_pdbfile.close()
                    ligand_positions, ligand_topology, ligand_md_topology = ligand_pdb.positions, ligand_pdb.topology, md.Topology.from_openmm(
                        ligand_pdb.topology)
                    ligand_n_atoms = ligand_md_topology.n_atoms

            elif isinstance(ligand_input, oechem.OEMol): # oemol object
                molecules.append(Molecule.from_openeye(ligand_input, allow_undefined_stereo=False))
                ligand_positions, ligand_topology = extractPositionsFromOEMol(ligand_input),  forcefield_generators.generateTopologyFromOEMol(ligand_input)
                ligand_md_topology = md.Topology.from_openmm(ligand_topology)
                ligand_n_atoms = ligand_md_topology.n_atoms

            else:
                _logger.warning(f'ligand filetype not recognised. Please provide a path to a .pdb or .sdf file')
                return

            # Now create a complex
            complex_md_topology = protein_md_topology.join(ligand_md_topology)
            complex_topology = complex_md_topology.to_openmm()
            complex_positions = unit.Quantity(np.zeros([protein_n_atoms + ligand_n_atoms, 3]), unit=unit.nanometers)
            complex_positions[:protein_n_atoms, :] = protein_positions
            complex_positions[protein_n_atoms:, :] = ligand_positions

        # Now for a system_generator
        self.system_generator = SystemGenerator(forcefields=forcefield_files,
                                                barostat=barostat,
                                                forcefield_kwargs=forcefield_kwargs,
                                                periodic_forcefield_kwargs=periodic_forcefield_kwargs,
                                                nonperiodic_forcefield_kwargs=nonperiodic_forcefield_kwargs,
                                                small_molecule_forcefield=small_molecule_forcefields,
                                                molecules=molecules,
                                                cache=None)

        # Solvate apo and complex...
        apo_input = list(self._solvate(protein_topology, protein_positions, water_model, phase, ionic_strength, apo_box_dimensions))
        inputs = [apo_input]
        if ligand_input:
            inputs.append(self._solvate(complex_topology, complex_positions, water_model, phase, ionic_strength, complex_box_dimensions))

        geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                                use_sterics=False,
                                                n_bond_divisions=100,
                                                n_angle_divisions=180,
                                                n_torsion_divisions=360,
                                                verbose=True,
                                                storage=None,
                                                bond_softening_constant=1.0,
                                                angle_softening_constant=1.0,
                                                neglect_angles = False,
                                                use_14_nonbondeds = True)


        # Run pipeline...
        htfs = []
        for (top, pos, sys) in inputs:
            point_mutation_engine = PointMutationEngine(wildtype_topology=top,
                                                                 system_generator=self.system_generator,
                                                                 chain_id=mutation_chain_id, # Denote the chain id allowed to mutate (it's always a string variable)
                                                                 max_point_mutants=1,
                                                                 residues_allowed_to_mutate=[mutation_residue_id], # The residue ids allowed to mutate
                                                                 allowed_mutations=[(mutation_residue_id, proposed_residue)], # The residue ids allowed to mutate with the three-letter code allowed to change
                                                                 aggregate=True) # Always allow aggregation

            topology_proposal = point_mutation_engine.propose(sys, top)

            # Only validate energy bookkeeping if the WT and proposed residues do not involve rings
            old_res = [res for res in top.residues() if res.id == mutation_residue_id][0]
            validate_bool = False if old_res.name in ring_amino_acids or proposed_residue in ring_amino_acids else True
            new_positions, logp_proposal = geometry_engine.propose(topology_proposal, pos, beta,
                                                                   validate_energy_bookkeeping=validate_bool)
            logp_reverse = geometry_engine.logp_reverse(topology_proposal, new_positions, pos, beta,
                                                        validate_energy_bookkeeping=validate_bool)

            if rxn_field:
                factory = RxnHybridTopologyFactory
            else:
                if repartitioned_endstate is None:
                    factory = HybridTopologyFactory
                elif repartitioned_endstate in [0, 1]:
                    factory = RepartitionedHybridTopologyFactory

            forward_htf = factory(topology_proposal=topology_proposal,
                                  current_positions=pos,
                                  new_positions=new_positions,
                                  use_dispersion_correction=False,
                                  functions=None,
                                  softcore_alpha=None,
                                  bond_softening_constant=1.0,
                                  angle_softening_constant=1.0,
                                  soften_only_new=False,
                                  neglected_new_angle_terms=[],
                                  neglected_old_angle_terms=[],
                                  softcore_LJ_v2=True,
                                  softcore_electrostatics=True,
                                  softcore_LJ_v2_alpha=0.85,
                                  softcore_electrostatics_alpha=0.3,
                                  softcore_sigma_Q=1.0,
                                  interpolate_old_and_new_14s=flatten_exceptions,
                                  omitted_terms=None,
                                  endstate=repartitioned_endstate,
                                  flatten_torsions=flatten_torsions,
                                  scale_regions=[scale_region])

            if not topology_proposal.unique_new_atoms:
                assert geometry_engine.forward_final_context_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.forward_final_context_reduced_potential})"
                assert geometry_engine.forward_atoms_with_positions_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's forward atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.forward_atoms_with_positions_reduced_potential})"
            else:
                added_valence_energy = geometry_engine.forward_final_context_reduced_potential - geometry_engine.forward_atoms_with_positions_reduced_potential

            if not topology_proposal.unique_old_atoms:
                assert geometry_engine.reverse_final_context_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.reverse_final_context_reduced_potential})"
                assert geometry_engine.reverse_atoms_with_positions_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.reverse_atoms_with_positions_reduced_potential})"
                subtracted_valence_energy = 0.0
            else:
                subtracted_valence_energy = geometry_engine.reverse_final_context_reduced_potential - geometry_engine.reverse_atoms_with_positions_reduced_potential


            if conduct_endstate_validation and repartitioned_endstate is None:
                zero_state_error, one_state_error = validate_endstate_energies(forward_htf._topology_proposal, forward_htf, added_valence_energy, subtracted_valence_energy, beta=beta, ENERGY_THRESHOLD=ENERGY_THRESHOLD)
                if zero_state_error > ENERGY_THRESHOLD:
                    _logger.warning(f"Reduced potential difference of the nonalchemical and alchemical Lambda = 0 state is above the threshold ({ENERGY_THRESHOLD}): {zero_state_error}")
                if one_state_error > ENERGY_THRESHOLD:
                    _logger.warning(f"Reduced potential difference of the nonalchemical and alchemical Lambda = 1 state is above the threshold ({ENERGY_THRESHOLD}): {one_state_error}")
            else:
                pass

            htfs.append(forward_htf)

        self.apo_htf = htfs[0]
        self.complex_htf = htfs[1] if ligand_input else None

In [61]:
solvent_delivery = PointMutationExecutor2("../../input/mmc2_barstar.pdb",
                        '1',
                        "42",
                        "ALA",
                        ligand_input="../../input/mmc2_barnase.pdb",
                        ionic_strength=0.05*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True, 
                        conduct_endstate_validation=False,
                        rxn_field=True
                       )

2021-07-14 16:40:01,938:(267.65s):setup:solvating at 0.05 M using tip3p
2021-07-14 16:40:13,227:(11.29s):setup:solvating at 0.05 M using tip3p
2021-07-14 16:40:36,279:(23.05s):proposal_generator:	Conducting polymer point mutation proposal...
2021-07-14 16:40:47,810:(11.53s):proposal_generator:Using matching_criterion to chose best atom map
2021-07-14 16:40:47,829:(0.02s):proposal_generator:Scaffold has symmetry of 0
2021-07-14 16:40:47,845:(0.02s):proposal_generator:Two molecules are not similar to have a common scaffold
2021-07-14 16:40:47,848:(0.00s):proposal_generator:Proceeding with direct mapping of molecules, but please check atom mapping and the geometry of the ligands.
2021-07-14 16:40:47,853:(0.01s):proposal_generator:len [{7: 9}, {8: 9}, {9: 9}, {7: 10}, {8: 10}, {9: 10}, {7: 11}, {8: 11}, {9: 11}, {7: 12}, {8: 12}, {9: 12}, {7: 13}, {8: 13}, {9: 13}]
2021-07-14 16:40:47,856:(0.00s):proposal_generator:{7: 9}
2021-07-14 16:40:47,858:(0.00s):proposal_generator:{8: 9}
2021-07-14

conducting subsequent work with the following platform: CUDA


2021-07-14 16:41:17,352:(1.76s):geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


2021-07-14 16:41:19,178:(1.83s):geometry:There are 4 new atoms
2021-07-14 16:41:19,191:(0.01s):geometry:	reduced angle potential = 0.05740176674496467.
2021-07-14 16:41:19,748:(0.56s):geometry:	reduced angle potential = 0.0493543421430842.
2021-07-14 16:41:20,286:(0.54s):geometry:	reduced angle potential = 0.19011523502542316.
2021-07-14 16:41:20,836:(0.55s):geometry:	reduced angle potential = 0.9835614087005569.
2021-07-14 16:41:22,296:(1.46s):geometry:	beginning construction of no_nonbonded final system...
2021-07-14 16:41:22,300:(0.00s):geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
2021-07-14 16:41:25,849:(3.55s):geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
2021-07-14 16:41:25,854:(0.00s):geometry:	there are 733 bond forces in the no-nonbonded final system
2021-07-14 16:41

...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-07-14 16:46:53,604:(4.12s):geometry:total reduced energy added from growth system: -62.3900963283472
2021-07-14 16:46:53,608:(0.00s):geometry:final reduced energy 4730.077774354767
2021-07-14 16:46:53,610:(0.00s):geometry:sum of energies: 4730.07777434895
2021-07-14 16:46:53,613:(0.00s):geometry:magnitude of difference in the energies: 5.817554438181105e-09
2021-07-14 16:46:53,615:(0.00s):geometry:Final logp_proposal: -10174.77849549996


added energy components: [('CustomBondForce', 0.4083956553523612), ('CustomAngleForce', 1.622079514297996), ('CustomTorsionForce', 17.503137529087592), ('CustomBondForce', -81.92370902708515)]


2021-07-14 16:46:53,944:(0.33s):relative:*** Generating RxnHybridTopologyFactory ***
2021-07-14 16:46:53,951:(0.01s):relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
2021-07-14 16:46:58,190:(4.24s):relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
2021-07-14 16:46:58,207:(0.02s):relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
2021-07-14 16:46:58,211:(0.00s):relative:No unknown forces.
2021-07-14 16:46:58,214:(0.00s):relative:Nonbonded method to be used (i.e. from old system): 4
2021-07-14 16:46:58,219:(0.00s):relative:Adding and mapping old atoms to hybrid system...
2021-07-14 16:46:59,174:(0.96s):relative:Adding and mapping new atoms to hybrid system...
2021-07-14 16:46:59,177:(0.00s):relative:scale_templates: [['nonscale_lambda', 'scale_lambda_0', 'i

2021-07-14 16:47:10,984:(0.02s):relative:hybrid_index_pair (2243, 2242, 2244, 2253) was not found in the new_term_collector, but (2242, 2243, 2244, 2253) has the same atoms and terms, so (2242, 2243, 2244, 2253) will be removed from the new term collector
2021-07-14 16:47:11,004:(0.02s):relative:hybrid_index_pair (2270, 2277, 2272, 2278) was not found in the new_term_collector, but (2270, 2278, 2272, 2277) has the same atoms and terms, so (2270, 2278, 2272, 2277) will be removed from the new term collector
2021-07-14 16:47:11,026:(0.02s):relative:hybrid_index_pair (2286, 2288, 2287, 2289) was not found in the new_term_collector, but (2288, 2287, 2286, 2289) has the same atoms and terms, so (2288, 2287, 2286, 2289) will be removed from the new term collector
2021-07-14 16:47:11,049:(0.02s):relative:hybrid_index_pair (2436, 2435, 2434, 2437) was not found in the new_term_collector, but (2434, 2436, 2435, 2437) has the same atoms and terms, so (2434, 2436, 2435, 2437) will be removed from

In [62]:
pickle.dump(solvent_delivery.get_apo_htf(), open("T42A_apo_scale_region.pickle", "wb" ))
pickle.dump(solvent_delivery.get_complex_htf(), open("T42A_complex_scale_region.pickle", "wb" ))


#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [63]:
# Load htf
with open("T42A_complex_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [64]:
test_bond_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 189.69678123582867), ('HarmonicAngleForce', 791.7783222646883), ('PeriodicTorsionForce', 3830.526380065008), ('NonbondedForce', -26748.767996075327), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 189.69678123582867), ('CustomAngleForce', 978.4710627815534), ('CustomTorsionForce', 3830.526548711567), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [65]:
# Load htf
with open("T42A_complex_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [66]:
test_bond_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 189.30454871026686), ('HarmonicAngleForce', 976.8489832672554), ('PeriodicTorsionForce', 3823.8766942707352), ('NonbondedForce', -26317.613716400156), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 189.30454871026686), ('CustomAngleForce', 978.4710627815534), ('CustomTorsionForce', 3830.526548711567), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


In [ ]:
# TO DO: actually test the scaling of the energies matches